## tensorboard 

## tf.reset_default_graph()

In [20]:
# Lab 9 XOR
import tensorflow as tf
import numpy as np

tf.set_random_seed(777)  # for reproducibility
learning_rate = 0.01

x_data = [[0, 0],
          [0, 1],
          [1, 0],
          [1, 1]]
y_data = [[0],
          [1],
          [1],
          [0]]
x_data = np.array(x_data, dtype=np.float32)
y_data = np.array(y_data, dtype=np.float32)

X = tf.placeholder(tf.float32, [None, 2], name='x-input')
Y = tf.placeholder(tf.float32, [None, 1], name='y-input')

with tf.name_scope("layer1"):
    W1 = tf.Variable(tf.random_normal([2, 2]), name='weight1')
    b1 = tf.Variable(tf.random_normal([2]), name='bias1')

x_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
y_data = np.array([[0], [1], [1], [0]], dtype=np.float32)

X = tf.placeholder(tf.float32, [None, 2], name="x")
Y = tf.placeholder(tf.float32, [None, 1], name="y")

with tf.name_scope("Layer1"):
    W1 = tf.Variable(tf.random_normal([2, 2]), name="weight_1")
    b1 = tf.Variable(tf.random_normal([2]), name="bias_1")
    layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)

    w1_hist = tf.summary.histogram("weights1", W1)
    b1_hist = tf.summary.histogram("biases1", b1)
    layer1_hist = tf.summary.histogram("layer1", layer1)
    tf.summary.histogram("W1", W1)
    tf.summary.histogram("b1", b1)
    tf.summary.histogram("Layer1", layer1)


with tf.name_scope("layer2"):
    W2 = tf.Variable(tf.random_normal([2, 1]), name='weight2')
    b2 = tf.Variable(tf.random_normal([1]), name='bias2')
with tf.name_scope("Layer2"):
    W2 = tf.Variable(tf.random_normal([2, 1]), name="weight_2")
    b2 = tf.Variable(tf.random_normal([1]), name="bias_2")
    hypothesis = tf.sigmoid(tf.matmul(layer1, W2) + b2)

    w2_hist = tf.summary.histogram("weights2", W2)
    b2_hist = tf.summary.histogram("biases2", b2)
    hypothesis_hist = tf.summary.histogram("hypothesis", hypothesis)
    tf.summary.histogram("W2", W2)
    tf.summary.histogram("b2", b2)
    tf.summary.histogram("Hypothesis", hypothesis)

# cost/loss function
with tf.name_scope("cost"):
    cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) *
                           tf.log(1 - hypothesis))
    cost_summ = tf.summary.scalar("cost", cost)
with tf.name_scope("Cost"):
    cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
    tf.summary.scalar("Cost", cost)

with tf.name_scope("train"):
    train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
with tf.name_scope("Train"):
    train = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

# Accuracy computation
# True if hypothesis>0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
accuracy_summ = tf.summary.scalar("accuracy", accuracy)
tf.summary.scalar("accuracy", accuracy)

# tf.reset_default_graph() # For Tensorboard with iPython

# Launch graph
with tf.Session() as sess:
    # tensorboard --logdir=./logs/xor_logs
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./logs/xor_logs_r0_01")
    writer.add_graph(sess.graph)  # Show the graph
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in range(1001):
        summary, _ = sess.run([merged_summary, train], feed_dict={X: x_data, Y: y_data})
        _, summary, cost_val = sess.run(
            [train, merged_summary, cost], feed_dict={X: x_data, Y: y_data}
        )
        writer.add_summary(summary, global_step=step)

        if step % 100 == 0:
            print(step, sess.run(cost, feed_dict={
                  X: x_data, Y: y_data}), sess.run([W1, W2]))
            print(step, cost_val)

    # Accuracy report
    h, c, a = sess.run([hypothesis, predicted, accuracy],
                       feed_dict={X: x_data, Y: y_data})
    print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)
    
    h, p, a = sess.run(
        [hypothesis, predicted, accuracy], feed_dict={X: x_data, Y: y_data}
    )

    print(f"\nHypothesis:\n{h} \nPredicted:\n{p} \nAccuracy:\n{a}")


    
    
'''
Hypothesis:  [[  6.13103184e-05]
 [  9.99936938e-01]
 [  9.99950767e-01]
 [  5.97514772e-05]]
Correct:  [[ 0.]
 [ 1.]
 [ 1.]
 [ 0.]]
Accuracy:  1.0
'''
    

"""
Hypothesis:
[[6.1310326e-05]
 [9.9993694e-01]
 [9.9995077e-01]
 [5.9751470e-05]] 
Predicted:
[[0.]
 [1.]
 [1.]
 [0.]] 
Accuracy:
1.0
"""

0 0.87062055 [array([[ 1.365222  , -0.76297426],
       [-0.6782276 , -1.1218139 ]], dtype=float32), array([[-0.06376155],
       [ 0.62445265]], dtype=float32)]
0 0.8766235
100 0.67181724 [array([[ 2.925827  , -0.51334774],
       [ 1.5143838 , -0.5190806 ]], dtype=float32), array([[0.5705866],
       [1.0302172]], dtype=float32)]
100 0.67220414
200 0.55128884 [array([[ 5.919449 , -1.289435 ],
       [ 4.4190392, -1.0382532]], dtype=float32), array([[1.9122691 ],
       [0.72379637]], dtype=float32)]
200 0.5518017
300 0.4172608 [array([[ 7.5688725, -2.379151 ],
       [ 6.096115 , -2.274029 ]], dtype=float32), array([[2.7314065],
       [1.8562661]], dtype=float32)]
300 0.41825995
400 0.23671898 [array([[ 8.617617 , -4.0252295],
       [ 7.173408 , -3.973245 ]], dtype=float32), array([[3.484123 ],
       [3.4359655]], dtype=float32)]
400 0.23737167
500 0.1454837 [array([[ 9.316678 , -5.0336742],
       [ 7.8883386, -4.9852333]], dtype=float32), array([[4.330538],
       [4.453612]], d

'\nHypothesis:\n[[6.1310326e-05]\n [9.9993694e-01]\n [9.9995077e-01]\n [5.9751470e-05]] \nPredicted:\n[[0.]\n [1.]\n [1.]\n [0.]] \nAccuracy:\n1.0\n'

In [1]:
import tensorflow as tf
import numpy as np

tf.set_random_seed(123)  # for reproducibility
learning_rate = 0.1

x_data = [[0, 0],
          [0, 1],
          [1, 0],
          [1, 1]]
y_data = [[0],
          [1],
          [1],
          [0]]

x_data = np.array(x_data, dtype=np.float32)
y_data = np.array(y_data, dtype=np.float32)

X = tf.placeholder(tf.float32, [None, 2])
Y = tf.placeholder(tf.float32, [None, 1])

conn1  = 10
conn2  = 10
conn3  = 10
connR  = 1

W1 = tf.Variable(tf.random_normal([2, conn1]), name='weight1')
b1 = tf.Variable(tf.random_normal([conn1]), name='bias1')
layer1 = tf.sigmoid(tf.matmul(X, W1) + b1, name='layer1')

W2 = tf.Variable(tf.random_normal([conn1, conn2]), name='weight2')
b2 = tf.Variable(tf.random_normal([conn2]), name='bias2')
layer2 = tf.sigmoid(tf.matmul(layer1, W2) + b2, name='layer2')

W3 = tf.Variable(tf.random_normal([conn2, conn3]), name='weight3')
b3 = tf.Variable(tf.random_normal([conn3]), name='bias3')
layer3 = tf.sigmoid(tf.matmul(layer2, W3) + b3, name='layer3')

W4 = tf.Variable(tf.random_normal([conn3, connR]), name='weight4')
b4 = tf.Variable(tf.random_normal([connR]), name='bias4')
hypothesis = tf.sigmoid(tf.matmul(layer3, W4) + b4, name='hypothesis')

# cost/loss function
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis), name='cost')

train = tf.train.GradientDescentOptimizer(learning_rate=learning_rate, name='train').minimize(cost)

# Accuracy computation
# True if hypothesis > 0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32), name='accuracy')

w1_hist = tf.summary.histogram("weight1", W1)
w2_hist = tf.summary.histogram("weight2", W2)
w3_hist = tf.summary.histogram("weight3", W3)
w4_hist = tf.summary.histogram("weight4", W4)

b1_hist = tf.summary.histogram("bias1", b1)
b2_hist = tf.summary.histogram("bias2", b2)
b3_hist = tf.summary.histogram("bias3", b3)
b4_hist = tf.summary.histogram("bias4", b4)

layer1_hist = tf.summary.histogram("layer1", layer1)
layer2_hist = tf.summary.histogram("layer2", layer2)
layer3_hist = tf.summary.histogram("layer3", layer3)
hypo_hist = tf.summary.histogram("hypothesis", hypothesis)

cost_summ = tf.summary.scalar("cost", cost)
accuracy_summ = tf.summary.scalar("accuracy", accuracy)

tf.reset_default_graph() # For Tensorboard with iPython
# Launch graph
with tf.Session() as sess:
    # tensorboard --logdir=./summaries/deep_nn_for_xor
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter('./summaries/deep_nn_for_xor')
    writer.add_graph(sess.graph)
    
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())
    
    for step in range(1001):
        print("step:", step)
        summ, _ = sess.run([merged, train], feed_dict={X: x_data, Y: y_data})
        writer.add_summary(summ, global_step=step)
        
        if step % 100 == 0:
            print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}), 
                  sess.run([W1, W2, W3, W4]))

    # Accuracy report
    h, c, a = sess.run([hypothesis, predicted, accuracy],
                       feed_dict={X: x_data, Y: y_data})
    print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)

Instructions for updating:
Colocations handled automatically by placer.
step: 0


TypeError: Cannot interpret feed_dict key as Tensor: Tensor Tensor("Placeholder:0", shape=(?, 2), dtype=float32) is not an element of this graph.

In [ ]:
w_summary = tf.summary.histogram("w", w)
b_summary = tf.summary.scalar("b", b)
z_summary = tf.summary.histogram("z", z)
cost_summary = tf.summary.scalar("cost", cost)
merged = tf.summary.merge_all()

init = tf.global_variables_initializer()
with tf.Session() as sess:
    writer = tf.summary.FileWriter("/home/dockeruser/logs", sess.graph)
    sess.run(init)
    for i in range(200):
        if i % 10 == 0:
            _, cost_, summary = sess.run([train, cost, merged])
            print("{:4d} : {:.5f} ".format(i, cost_))
        else:
            _, summary = sess.run([train, merged])
        writer.add_summary(summary, i)
    y_pred = sess.run(tf.sign(z))

In [2]:
import tensorflow as tf

a = tf.constant(3.0)
b = tf.constant(5.0)
c = a * b

# tensorboard에 point라는 이름으로 표시됨
c_summary = tf.summary.scalar('point', c)
merged = tf.summary.merge_all()

with tf.Session() as sess:
    writer = tf.summary.FileWriter('./sample_1', sess.graph)

    result = sess.run([merged])
#    result = sess.run(merged)  # error
    sess.run(tf.global_variables_initializer())

    writer.add_summary(result[0]) 

In [ ]:
import tensorflow as tf

a = tf.constant(3, name='aa')
b = tf.constant(7, name='b')
c = tf.constant(7, name='c')
# e = tf.constant(7, name='e')
d = a * b + c

# tensorboard에 point라는 이름으로 표시됨
c_summary = tf.summary.scalar('point', c)
merged = tf.summary.merge_all()

with tf.Session() as sess:    
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('C:/Users/gwihy/ML_01/board/sample', sess.graph)
    sess.run(tf.global_variables_initializer())
    result = sess.run([merged])
    writer.add_summary(result[0]) 